In [1]:
from gensim.models import Word2Vec
import os
import pandas as pd

In [2]:
path = './data/'
def readdata(path):
    list_file = os.listdir(path)
    data = pd.DataFrame()
    for filename in list_file:
        data = pd.concat([data, pd.read_csv(os.path.join(path, filename), sep = ',')])
        
    return data.Review, data.Label

In [4]:
reviews, labels = readdata(path)
input_gensim = []
for review in reviews:
    input_gensim.append(review.split())
    
model = Word2Vec(input_gensim, vector_size=128, window=5, min_count=0, workers=4, sg=1)
model.wv.save("word.model")

In [6]:
import gensim.models.keyedvectors as word2vec
import numpy as np

model_embedding = word2vec.KeyedVectors.load('./word.model')

word_labels = list(model_embedding.key_to_index.keys())
max_seq = 200
embedding_size = model_embedding.vector_size  # Sử dụng kích thước vector thực tế từ mô hình

def comment_embedding(comment):
    matrix = np.zeros((max_seq, embedding_size))
    words = comment.split()
    lencmt = len(words)

    for i in range(min(max_seq, lencmt)):
        word = words[i]
        if word in model_embedding.key_to_index:
            matrix[i] = model_embedding[word]
    
    # Nếu comment ngắn hơn max_seq, phần còn lại của matrix sẽ là 0
    return matrix

In [7]:
# Ví dụ sử dụng
example_comment = "Đây là một ví dụ comment"
embedded_comment = comment_embedding(example_comment)
print(f"Shape of embedded comment: {embedded_comment.shape}")

Shape of embedded comment: (200, 128)


In [9]:
from tqdm import tqdm
train_data = []
label_data = []

for x in tqdm(reviews):
    train_data.append(comment_embedding(x))
train_data = np.array(train_data)

for y in tqdm(labels):
    label_ = np.zeros(3)
    try:
        label_[int(y)] = 1
    except:
        label_[0] = 1
    label_data.append(label_)

100%|██████████| 9727/9727 [00:00<?, ?it/s]


In [11]:
import numpy as np
from tensorflow.keras import layers
from tensorflow import keras 
import tensorflow as tf
from keras.preprocessing import sequence

sequence_length = 200
embedding_size = 128
num_classes = 3
filter_sizes = 3
num_filters = 150
epochs = 50
batch_size = 30
learning_rate = 0.01
dropout_rate = 0.5

In [13]:
x_train = train_data.reshape(train_data.shape[0], sequence_length, embedding_size, 1).astype('float32')
y_train = np.array(label_data)

# Define model
model = keras.Sequential()
model.add(layers.Convolution2D(num_filters, (filter_sizes, embedding_size),
                        padding='valid',
                        input_shape=(sequence_length, embedding_size, 1), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(198, 1)))
model.add(layers.Dropout(dropout_rate))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))
# Train model
adam = tf.keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)               │ (None, 198, 1, 150)    │        57,750 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 1, 1, 150)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1, 1, 150)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 150)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        19,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 77,465 (302.60 KB)

 Trainable params: 77,465 (302.60 KB)

 Non-trainable params: 0 (0.00 B)

None


In [14]:
model.fit(x = x_train[:7000], y = y_train[:7000], batch_size = batch_size, verbose=1, epochs=epochs, validation_data=(x_train[:3000], y_train[:3000]))

model.save('models.h5')

Epoch 1/50
234/234 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.5405 - loss: 0.9373 - val_accuracy: 0.7663 - val_loss: 0.6350
Epoch 2/50
234/234 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.7219 - loss: 0.6958 - val_accuracy: 0.7157 - val_loss: 0.7153
Epoch 3/50
234/234 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.7358 - loss: 0.6571 - val_accuracy: 0.7863 - val_loss: 0.5570
Epoch 4/50
234/234 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.7575 - loss: 0.6082 - val_accuracy: 0.8173 - val_loss: 0.4901
Epoch 5/50
234/234 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.7661 - loss: 0.5929 - val_accuracy: 0.7830 - val_loss: 0.5855
Epoch 6/50
234/234 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.7700 - loss: 0.5649 - val_accuracy: 0.7983 - val_loss: 0.5214
Epoch 7/50
234/234 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.7934 - loss: 0.5398 - val_accuracy: 0.8103 - val_loss: 0.5370
Epoch 8/50
234/234 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.7959 - loss: 0.5210 - val_accu

In [ ]:
import re
import underthesea

# Chuẩn hóa các từ viết tắt
abbreviation_dict = {
    'k': 'không',
    'ko': 'không',
    'k0': 'không',
    'bt': 'bình thường',
    'ntn': 'như thế nào',
    # Thêm các từ viết tắt khác nếu cần
}

# Hàm xử lý tiền xử lý văn bản
def pre_process(text):
    # 1. Chuyển về chữ thường
    text = text.lower()

    # 2. Thay thế các URL bằng nhãn 'link_spam'
    text = re.sub(r'http\S+|www\S+|https\S+', 'link_spam', text, flags=re.MULTILINE)

    # 3. Xử lý các trường hợp lặp âm tiết (ví dụ: Ngoooon quááááá)
    text = re.sub(r'(.)\1+', r'\1', text)  # Loại bỏ âm tiết lặp lại nhiều lần

    # 4. Chuẩn hóa các từ viết tắt
    words = text.split()
    text = ' '.join([abbreviation_dict.get(word, word) for word in words])

    # 5. Loại bỏ dấu câu và các ký tự đặc biệt
    text = re.sub(r'[^\w\s]', '', text)

    # 6. Loại bỏ số và các từ chỉ có 1 ký tự
    text = ' '.join([word for word in text.split() if len(word) > 1 and not word.isdigit()])

    # 7. Tách từ sử dụng Underthesea
    text = underthesea.word_tokenize(text, format="text")

    return text

# Ví dụ sử dụng
text = "đồ ăn ở đây vừa nhiều vừa ngon!!! Ngooon quááááá ơiiii! http://example.com"
text = pre_process(text)

# In ra kết quả tiền xử lý
print("Text sau tiền xử lý: ", text)

# Tiếp tục với việc nhúng (embedding) và dự đoán như đã có
maxtrix_embedding = np.expand_dims(comment_embedding(text), axis=0)
maxtrix_embedding = np.expand_dims(maxtrix_embedding, axis=3)

result = model.predict(maxtrix_embedding)
result = np.argmax(result)
print("Label predict: ", result)

In [ ]:
# Danh sách các mẫu
samples = [
    "Thật tuyệt vời!",   # positive
    "Mọi thứ đều ổn.",   # neutral
    "Tôi không thích nó.", # negative
    "Đồ ăn ngon quá!",   # positive
    "Có thể chấp nhận được.", # neutral
    "Quá tệ!",           # negative
    "Mọi thứ thật tuyệt vời!", # positive
    "Chỉ bình thường thôi.", # neutral
    "Tôi rất thất vọng."  # negative
]

# Tiền xử lý và dự đoán cho từng mẫu
for text in samples:
    processed_text = pre_process(text)
    print("Text sau tiền xử lý: ", processed_text)

    # Nhúng (embedding) và dự đoán
    maxtrix_embedding = np.expand_dims(comment_embedding(processed_text), axis=0)
    maxtrix_embedding = np.expand_dims(maxtrix_embedding, axis=3)

    result = model.predict(maxtrix_embedding)
    label = np.argmax(result)
    print("Label predict: ", label) # 0:neutral, 1:positive, 2:negative